```
sudo docker run --privileged --user carla --gpus all --net=host -e DISPLAY=$DISPLAY carlasim/carla-add:0.9.13 /bin/bash ./CarlaUE4.sh -world-port=12321 -RenderOffScreen
```

In [1]:
import carla

import json
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
from env.CarlaEnv import CarlaEnv
from utils.VideoRecorder import VideoRecorder

In [3]:
with open('./cfg/weather.json', 'r', encoding='utf8') as fff:
    weather_params = json.load(fff)
with open('./cfg/scenario.json', 'r', encoding='utf8') as fff:
    scenario_params = json.load(fff)

In [4]:
fps = 20

In [5]:
carla_env = CarlaEnv(
    weather_params=weather_params,
    scenario_params=scenario_params,
    selected_weather="L1",
    selected_scenario="jaywalk",
    carla_rpc_port=12321,
    carla_tm_port=18935,
    carla_timeout=8,
    perception_type="dvs_frame",
    num_cameras=5,
    rl_image_size=128,
    fov=60,
    max_fps=fps,
    min_fps=fps,
    max_episode_steps=1000,
    frame_skip=1,
)

carla env reset done.


In [6]:
throttles = []
brakes = []
velocitys = []

In [7]:
max_episode_num = 100
max_step_num = 200

In [8]:
video = VideoRecorder("./video", fps=fps)

In [ ]:
for one_episode in range(max_episode_num):
    
    try:
        print("starting episode:", one_episode+1)
        carla_env.reset()

        throttle = []
        brake = []
        velocity = []

        video.init(True) 
        for one_step in range(max_step_num):

            if one_step <= 100:
                action = [0, 0.7]
            else:
                action = [0, -0.3]

            obs, reward, done, info = carla_env.step(action)

            video.record(obs, carla_env.vehicle)


            print(f"\rframe: {carla_env.frame}", end="")

            throttle.append(carla_env.vehicle.get_control().throttle)
            brake.append(carla_env.vehicle.get_control().brake)
            velocity.append(carla_env.vehicle.get_velocity().x)
    except Exception as e:
        print(e)
        
    video.save(f"test-{one_episode+1}")
    print(f"\nsave video done.\n")

    throttles.append(throttle)
    brakes.append(brake)
    velocitys.append(velocity)
    

starting episode: 1
carla env reset done.
frame: 379
save video done.

starting episode: 2


In [ ]:
times = list(range(len(velocity)))

plt.plot(times, velocity, label="velocity")
plt.plot(times, throttle, label="throttle")
plt.plot(times, brake, label="brake")
plt.legend()